<a href="https://colab.research.google.com/github/sergejhorvat/Tensorflow2.0_Udemy/blob/master/Preparing_a_TensorFlow_model_for_a_mobile_device_with_TensorFlow_Lite.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Stage 1: Installing dependencies and setting up the environment

Using TF 1.13.1 becaouse in TF 2 there is not yet TF Lite library.

In [1]:
!pip install tensorflow-gpu==1.13.1

     |████████████████████████████████| 345.2MB 94kB/s 
     |████████████████████████████████| 3.2MB 47.2MB/s 
     |████████████████████████████████| 368kB 44.0MB/s 
ERROR: tensorflow 1.14.0 has requirement tensorboard<1.15.0,>=1.14.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.14.0 has requirement tensorflow-estimator<1.15.0rc0,>=1.14.0rc0, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0


## Stage 2: Importing project dependencies

In [4]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.datasets import fashion_mnist

tf.__version__

'1.13.1'

## Stage 3: Dataset preprocessing

### Loading the FashionMNIST dataset

In [5]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


### Image normalization

In [0]:
X_train = X_train / 255.
X_test = X_test / 255.

### Reshaping of the dataset

In [7]:
X_train.shape

(60000, 28, 28)

In [0]:
# -1 takes all images in array
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [12]:
X_train.shape

(60000, 784)

## Stage 4: Building a model

### Defining the model

In [0]:
model = tf.keras.models.Sequential()

### Building the model

In [14]:
model.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
model.add(tf.keras.layers.Dropout(rate=0.2))
model.add(tf.keras.layers.Dense(units=10, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


### Compiling the model

In [0]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Training the model

In [17]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 6s 100us/sample - loss: 0.5297 - sparse_categorical_accuracy: 0.8128
Epoch 2/5
60000/60000 [==============================] - 5s 84us/sample - loss: 0.3969 - sparse_categorical_accuracy: 0.8571
Epoch 3/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.3681 - sparse_categorical_accuracy: 0.8668
Epoch 4/5
60000/60000 [==============================] - 5s 85us/sample - loss: 0.3451 - sparse_categorical_accuracy: 0.8728
Epoch 5/5
60000/60000 [==============================] - 5s 86us/sample - loss: 0.3287 - sparse_categorical_accuracy: 0.8785


### Evaluating the model

In [18]:
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {}".format(test_accuracy))

10000/10000 [==============================] - 1s 50us/sample - loss: 0.3554 - sparse_categorical_accuracy: 0.8715
Test accuracy: 0.8715000152587891


## Stage 5: Converting the model into TensorFlow Lite version

### Saving the model

In [0]:
model_name = 'fashion_mobile_model.h5'
tf.keras.models.save_model(model, model_name)

### Creating the TFLite Converter

In [21]:
converter = tf.lite.TFLiteConverter.from_keras_model_file(model_name)

INFO:tensorflow:Froze 4 variables.
INFO:tensorflow:Converted 4 variables to const ops.


### Converting the model

In [0]:
tflite_model = converter.convert()

### Saving the TFLite version of the model

In [0]:
with open("tf_model.tflite", "wb") as f:
  f.write(tflite_model)